In [ ]:
#external packages

#! pip install BeautifulSoup
#! pip install word2number
#! pip install pyspellchecker
#! pip install transformers

In [ ]:
#load the dataset
import csv

data = []
labels = []

file = open("IMDB Dataset.csv", 'r', encoding='utf8')
reader = csv.reader(file)

for line in reader:
    if line[1] == 'positive':
        data.append(line[0])
        labels.append('optimistic')
    elif line[1] == 'negative':
        data.append(line[0])
        labels.append('pessimistic')

In [ ]:
# preprocessing

from bs4 import BeautifulSoup
from word2number import w2n
from spellchecker import SpellChecker
import re

def remove_numbers(text):
    def is_written_number(word):
        try:
            w2n.word_to_num(word)
            return True
        except ValueError:
            return False

    # Remove numerical digits
    cleaned_text = re.sub(r'\d+', '', text)
    # Remove written numbers
    cleaned_text = re.sub(r'\b([a-zA-Z]+(-[a-zA-Z]+)?)\b', lambda x: '' if is_written_number(x.group()) else x.group(), cleaned_text)
    
    return cleaned_text

# does not work perfectly because of punctuation, splitting is done by ' '
def fix_spelling(text):
    spell = SpellChecker()
    return ' '.join([spell.correction(word) if spell.correction(word) != None else word for word in text.split()])

def condense_spaces(text):
    return re.sub(r'\s+', ' ', text)

clean_data = []
cnt = 0
for text in data:
    #remove html tags
    soup = BeautifulSoup(text, "html.parser")
    processed_text = soup.get_text()

    #remove numbers
    processed_text = remove_numbers(processed_text) 

    #remove sequences of spaces
    processed_text = condense_spaces(processed_text)

    #lowercase
    processed_text = processed_text.lower()

    #fix spelling issues
    #fixed_spelling = fix_spelling(lowercase_text)

    clean_data.append(processed_text)
    print(cnt)
    cnt += 1

In [ ]:
# tokenization
from transformers import BertTokenizer

tokenized_data = []
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

for data in clean_data:
    tokenized_data.append(tokenizer.tokenize(data))
 
print(len(tokenized_data), len(labels))